In [1]:
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random

import PIL
import matplotlib.pyplot as plt
%matplotlib inline
import cv2
import os
import random
import math
import numpy as np
import h5py
import matplotlib.pyplot as plt
import tensorflow as tf

from numpy import save
from numpy import load

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow import keras
from tensorflow.keras import Input, Model

from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPool2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.utils import to_categorical

In [2]:
training_path = "G:/Desktop/intel image classification/seg_train/seg_train/"
testing_path = "G:/Desktop/intel image classification/seg_test/seg_test/"

In [3]:
generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255,
                                                           zca_whitening=True,
                                                           rotation_range=25,
                                                           width_shift_range=0.15,
                                                           horizontal_flip=True,
                                                            validation_split = 0.1
                                                           )
training_instances = generator.flow_from_directory(training_path, 
                                                   target_size=(224, 224),
                                                   batch_size=16,
                                                  subset='training')

dev_instances = generator.flow_from_directory(training_path, 
                                                   target_size=(224, 224),
                                                   batch_size=16,
                                                  subset='validation')

generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255)
test_instances = generator.flow_from_directory(testing_path,
                                               target_size=(224, 224),
                                               batch_size=16)

C:\Users\Dindar\anaconda3\lib\site-packages\keras_preprocessing\image\image_data_generator.py:336: UserWarning: This ImageDataGenerator specifies `zca_whitening`, which overrides setting of `featurewise_center`.
  warnings.warn('This ImageDataGenerator specifies '


Found 12632 images belonging to 6 classes.
Found 1402 images belonging to 6 classes.
Found 3000 images belonging to 6 classes.


In [4]:
base_model = tf.keras.applications.VGG19(weights="imagenet", include_top=True, input_shape=(224,224,3))

In [5]:
len(base_model.layers)

26

In [6]:
base_model.trainable = False

In [7]:
base_model.summary()

Model: "vgg19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [8]:
x = base_model.get_layer('fc2').output

prediction = Dense(6, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=prediction)

In [9]:
model.compile(loss="categorical_crossentropy", optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics=["accuracy"])

In [10]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [11]:
%%time
Batch_size=16
initial_epochs = 8
model_fit=model.fit(training_instances, 
                    steps_per_epoch=12632 // Batch_size,
                    epochs=initial_epochs,
                    validation_data=dev_instances
                   )

C:\Users\Dindar\anaconda3\lib\site-packages\keras_preprocessing\image\image_data_generator.py:716: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
C:\Users\Dindar\anaconda3\lib\site-packages\keras_preprocessing\image\image_data_generator.py:735: UserWarning: This ImageDataGenerator specifies `zca_whitening`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 789 steps, validate for 88 steps
Epoch 1/8


C:\Users\Dindar\anaconda3\lib\site-packages\keras_preprocessing\image\image_data_generator.py:716: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
C:\Users\Dindar\anaconda3\lib\site-packages\keras_preprocessing\image\image_data_generator.py:735: UserWarning: This ImageDataGenerator specifies `zca_whitening`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


789/789 [==============================] - 325s 412ms/step - loss: 0.7442 - accuracy: 0.7058 - val_loss: 0.6220 - val_accuracy: 0.7496
Epoch 2/8
789/789 [==============================] - 372s 472ms/step - loss: 0.5950 - accuracy: 0.7697 - val_loss: 0.5636 - val_accuracy: 0.7853
Epoch 3/8
789/789 [==============================] - 275s 349ms/step - loss: 0.5401 - accuracy: 0.7967 - val_loss: 0.5658 - val_accuracy: 0.7903
Epoch 4/8
789/789 [==============================] - 228s 290ms/step - loss: 0.5356 - accuracy: 0.8002 - val_loss: 0.4914 - val_accuracy: 0.8210
Epoch 5/8
789/789 [==============================] - 223s 282ms/step - loss: 0.5181 - accuracy: 0.8061 - val_loss: 0.4906 - val_accuracy: 0.8295
Epoch 6/8
789/789 [==============================] - 221s 281ms/step - loss: 0.5218 - accuracy: 0.8052 - val_loss: 0.5869 - val_accuracy: 0.7910
Epoch 7/8
789/789 [==============================] - 230s 291ms/step - loss: 0.5021 - accuracy: 0.8166 - val_loss: 0.4556 - val_accuracy: 0.

In [12]:
model.evaluate(dev_instances), model.evaluate(test_instances)

  ...
    to  
  ['...']
88/88 [==============================] - 25s 280ms/step - loss: 0.4744 - accuracy: 0.8345
  ...
    to  
  ['...']
188/188 [==============================] - 37s 196ms/step - loss: 0.4123 - accuracy: 0.8523


([0.47441510754552757, 0.8345221], [0.4122946983044769, 0.8523333])

In [13]:
%%time
initial_epochs1 = 8
total_epochs =  initial_epochs + initial_epochs1

history_fine = model.fit(training_instances,
                         steps_per_epoch=12632 // Batch_size,
                         epochs=total_epochs,
                         initial_epoch =  model_fit.epoch[-1],
                         validation_data=dev_instances)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 789 steps, validate for 88 steps
Epoch 8/16
789/789 [==============================] - 293s 371ms/step - loss: 0.4951 - accuracy: 0.8167 - val_loss: 0.6010 - val_accuracy: 0.7846
Epoch 9/16
789/789 [==============================] - 250s 317ms/step - loss: 0.4866 - accuracy: 0.8186 - val_loss: 0.6136 - val_accuracy: 0.7860
Epoch 10/16
357/789 [============>.................] - ETA: 2:09 - loss: 0.4541 - accuracy: 0.8263

KeyboardInterrupt: 

In [15]:
base_model.trainable = True

In [14]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [16]:
len(base_model.layers)

26

In [17]:
# Let's take a look to see how many layers are in the base model
print("Number of layers in the base model: ", len(base_model.layers))

# Fine-tune from this layer onwards
fine_tune_at = 23

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable =  False

Number of layers in the base model:  26


In [18]:
model.compile(loss="categorical_crossentropy", optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), metrics=["accuracy"])

In [20]:
%%time
fine_tune_epochs = 10
total_epochs =  initial_epochs + fine_tune_epochs

history_fine = model.fit(training_instances,
                         epochs=total_epochs,
                         initial_epoch =  model_fit.epoch[-1],
                         validation_data=dev_instances)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 790 steps, validate for 88 steps
Epoch 8/18
790/790 [==============================] - 249s 315ms/step - loss: 0.5970 - accuracy: 0.7788 - val_loss: 0.6814 - val_accuracy: 0.7696
Epoch 9/18
790/790 [==============================] - 255s 322ms/step - loss: 0.5302 - accuracy: 0.8150 - val_loss: 0.4180 - val_accuracy: 0.8588
Epoch 10/18
790/790 [==============================] - 264s 334ms/step - loss: 0.4230 - accuracy: 0.8452 - val_loss: 0.4361 - val_accuracy: 0.8452
Epoch 11/18
790/790 [==============================] - 252s 319ms/step - loss: 0.3716 - accuracy: 0.8653 - val_loss: 0.4965 - val_accuracy: 0.8352
Epoch 12/18
790/790 [==============================] - 268s 339ms/step - loss: 0.3486 - accuracy: 0.8740 - val_loss: 0.3888 - val_accuracy: 0.8623
Epoch 13/18
790/790 [==============================] - 253s 320ms/step - loss: 0.3227 - accuracy: 0.8809 - val_loss: 0.3843 - val_accuracy: 0.8616
Epoch 14/18
790/790 [======

In [21]:
base_model.trainable = True

In [22]:
# Let's take a look to see how many layers are in the base model
print("Number of layers in the base model: ", len(base_model.layers))

# Fine-tune from this layer onwards
fine_tune_at = 18

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable =  False

Number of layers in the base model:  26


In [23]:
model.compile(loss="categorical_crossentropy", optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), metrics=["accuracy"])

In [24]:
%%time
fine_tune_epochs1 = 13
total_epochs =  initial_epochs + fine_tune_epochs + fine_tune_epochs1

history_fine = model.fit(training_instances,
                         epochs=total_epochs,
                         initial_epoch =  history_fine.epoch[-1],
                         validation_data=dev_instances)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 790 steps, validate for 88 steps
Epoch 18/31


C:\Users\Dindar\anaconda3\lib\site-packages\keras_preprocessing\image\image_data_generator.py:716: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
C:\Users\Dindar\anaconda3\lib\site-packages\keras_preprocessing\image\image_data_generator.py:735: UserWarning: This ImageDataGenerator specifies `zca_whitening`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


790/790 [==============================] - 248s 314ms/step - loss: 0.4250 - accuracy: 0.8462 - val_loss: 0.3831 - val_accuracy: 0.8659
Epoch 19/31
790/790 [==============================] - 235s 298ms/step - loss: 0.3259 - accuracy: 0.8841 - val_loss: 0.3303 - val_accuracy: 0.8902
Epoch 20/31
790/790 [==============================] - 237s 300ms/step - loss: 0.2771 - accuracy: 0.8967 - val_loss: 0.3360 - val_accuracy: 0.8809
Epoch 21/31
790/790 [==============================] - 238s 302ms/step - loss: 0.2464 - accuracy: 0.9111 - val_loss: 0.3228 - val_accuracy: 0.8923
Epoch 22/31
790/790 [==============================] - 237s 300ms/step - loss: 0.2310 - accuracy: 0.9177 - val_loss: 0.3259 - val_accuracy: 0.8845
Epoch 23/31
790/790 [==============================] - 236s 298ms/step - loss: 0.2109 - accuracy: 0.9215 - val_loss: 0.3379 - val_accuracy: 0.8880
Epoch 24/31
790/790 [==============================] - 240s 303ms/step - loss: 0.1926 - accuracy: 0.9306 - val_loss: 0.3509 - val_

KeyboardInterrupt: 

In [25]:
model.evaluate(dev_instances), model.evaluate(test_instances)

  ...
    to  
  ['...']
88/88 [==============================] - 20s 227ms/step - loss: 0.3524 - accuracy: 0.8902
  ...
    to  
  ['...']
188/188 [==============================] - 37s 195ms/step - loss: 0.3276 - accuracy: 0.9043s - loss: 0.3239 - ac


([0.3523578751324253, 0.8901569], [0.32759657890555705, 0.90433335])

In [26]:
model.save('G:/Desktop/intel image classification/90%(VGG19).h5')